In [4]:
from officelib.xllib import *
from pywintypes import com_error
import officelib.const as xlc


In [24]:
def extract_data(ws):
    ur = ws.UsedRange
    fc = ur.Columns(1)

    data = ur.Value
    splitdata = []
    for d in data:
        frs = d[0]
        if not frs:
            splitdata.append(d)
            continue
        frs = frs.split("\n")
        frs = [f.strip() for f in frs]
        frs = [f for f in frs if f]
        for f in frs:
            new = (f,) + d[1:]
            splitdata.append(new)
    return splitdata


In [25]:


def remake_sheet(wb, name):
    try:
        ws2 = wb.Worksheets(name)
        ws2.Delete()
    except com_error:
        pass
    ws2 = wb.Worksheets.Add()
    ws2.Name = name
    return ws2

def calc_range(ws, data):
    # Data should be rectangular matrix - uneven row lengths
    # not supported!
    c1 = ws.Cells(1,1)
    c2 = c1.Offset(len(data), len(data[0]))
    paste_range = ws.Cells.Range(c1, c2)
    return paste_range

def format_data(paste_range):
    print("Updating Column Widths...")
    for c in paste_range.Columns:
        c.AutoFit()
        if c.ColumnWidth > 50:
            c.ColumnWidth=50
    paste_range.WrapText = True
    print("Updating Row Heights...")
    for r in paste_range.Rows:
        r.AutoFit()
    print("Updating Formatting...")
    paste_range.HorizontalAlignment = xlc.xlLeft
    paste_range.VerticalAlignment = xlc.xlTop
    paste_range.Font.Size = 9

def paste_data1(ws, splitdata):
    paste_range = calc_range(ws, splitdata)
    print("Pasting Data...")
    paste_range.Value = splitdata
    format_data(paste_range)

In [26]:
def paste_data2(ws, splitdata):
    data = [d[:7] for d in splitdata]
    paste_range = calc_range(ws, data)
    paste_range.Value = data
    format_data(paste_range)
    
    
def main(wb):
    ws=wb.Worksheets("Sheet1")
    data = extract_data(ws)

    with screen_lock(xl):
        ws2 = remake_sheet(wb, "FRS")
        paste_data1(ws2, data)

    with screen_lock(xl):
        ws3 = remake_sheet(wb, "ID Matrix")
        paste_data2(ws3, data)

In [27]:
xl = Excel()
wb = xl.ActiveWorkbook
main(wb)

Pasting Data...
Updating Column Widths...
Updating Row Heights...
Updating Formatting...
Done
Updating Column Widths...
Updating Row Heights...
Updating Formatting...
